In [16]:
import torch
from torch import nn
import torch.nn.functional as F

In [15]:
import timm
from transformers import DistilBertModel, DistilBertConfig

In [11]:
class ImageEncoder(nn.Module):
    def __init__(
        self, model_name, pretrained, trainable
    ):
        super().__init__()
        self.model = timm.create_model(
            model_name, pretrained, num_classes=0, global_pool="avg"
        )
        
        for p in self.model.parameters():
            p.requires_grad = trainable
    
    def forward(self, x):
        return self.model(x)

In [3]:
class TextEncoder(nn.Module):
    def __init__(self, model_name, pretrained, trainable):
        super().__init__()
        
        if pretrained:
            self.model = DistilBertModel.from_pretrained(model_name)
        else:
            self.model = DistilBertModel(config=DistilBertConfig())
        
        for p in self.model.parameters():
            p.requires_grad = trainable
        
        self.target_token_idx = 0
    
    def forward(self, input_ids, attention_mask):
        output = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        last_hidden_state = output.last_hidden_state

In [4]:
class ProjectionHead:
    def __init__(self, embedding_dim):
        pass

In [17]:
class CLIPModel(nn.Module):
    def __init__(
        self,
        temperature,
        image_embedding,
        text_embedding
    ):
        super().__init__()
        self.image_encoder = ImageEncoder()
        self.text_encoder = TextEncoder()
        self.image_projection = ProjectionHead(embedding_dim=image_embedding)
        self.text_projection = ProjectionHead(embedding_dim=text_embedding)
        self.temperature = temperature
    
    def forward(self, batch):
        
        image = batch["image"]
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        
        image_features = self.image_encoder(image)
        text_features = self.text_encoder(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        
        # getting image and text embeddings
        image_embeddings = self.image_projection(image_features)
        text_embeddings = self.text_projection(text_features)
        
        # calculate the loss

https://github.com/moein-shariatnia/OpenAI-CLIP